In [196]:
def argmax(lst): 
    return lst.index(max(lst))

def find_max(col, DATA_COUNT):
    max_num = -999
    for i in range(DATA_COUNT):
        if col[i] > max_num:
            max_num = col[i]
    return max_num

def find_min_pos(col, DATA_COUNT, skip_list, k):
    min_num = 100000
    min_idx = 100000
    
    for num in range(DATA_COUNT):

        toggle = False
        for idx in range(k):
            if num == skip_list[idx]:
                toggle = True
            
        if col[num] < min_num and toggle == False:
            min_num = col[num]
            min_idx = num
            
    for num in range(k):
        if skip_list[num] == -1:
            skip_list[num] = min_idx
            break
    return min_idx, skip_list

def find_max_pos(col, DATA_COUNT):
    max_num = -999
    pos = -999
    for i in range(DATA_COUNT):
        if col[i] > max_num:
            max_num = col[i]
            pos = i
    return max_num, pos

def find_min(col, DATA_COUNT):
    min_num = 100000
    for i in range(DATA_COUNT):
        if col[i] < min_num:
            min_num = col[i]
    return min_num

def sort_col(arr, DATA_COUNT):
    
    # 先讓 arr_sort 裡面全部都是 -100
    arr_sort = []
    arr_idx = []
    
    for i in range(DATA_COUNT):
        arr_sort.append(-100)
        arr_idx.append(-100)
        
    for count in range(DATA_COUNT):

        # 找到當前 arr 裡面的最大值 以及 他的 index
        max_number = -1000
        max_number_idx = -1000
        for i in range(DATA_COUNT):

            if arr[i] > max_number:
                max_number = arr[i]
                max_number_idx = i

        # 把他便最小，這樣之後就不會挑到他了
        arr[max_number_idx] = -1000

        # 把當前最大值存到 arr_sort 裡面
        arr_sort[count] = max_number
        arr_idx[count] = max_number_idx
    return arr_sort, arr_idx

def init_attributes(length):
    init_attr_0, init_attr_1, init_attr_2, init_attr_3, init_attr_4, init_attr_5, init_attr_6, init_attr_7, init_attr_8 = [], [], [], [], [], [], [], [], []
    for _ in range(length):
        init_attr_0.append(-999)   
        init_attr_1.append(-999)  
        init_attr_2.append(-999)  
        init_attr_3.append(-999)
        init_attr_4.append(-999)
        init_attr_5.append(-999)  
        init_attr_6.append(-999)  
        init_attr_7.append(-999)
        init_attr_8.append(-999)

    attributes = [init_attr_0, init_attr_1, init_attr_2, init_attr_3, init_attr_4, init_attr_5, init_attr_6, init_attr_7, init_attr_8]

    return attributes

from tqdm import tqdm

def map_into_train_dataset(train_dataset, num_1, num_2, num_3, num_4, AF_1, AF_2, AF_3, AF_4):
    
    counter = 0
    for idx in range(num_1):
        for col_num in range(9):
            train_dataset[col_num][counter] = AF_1[col_num][idx]
        counter = counter + 1

    for idx in range(num_2):
        for col_num in range(9):
            train_dataset[col_num][counter] = AF_2[col_num][idx]
        counter = counter + 1

    for idx in range(num_3):
        for col_num in range(9):
            train_dataset[col_num][counter] = AF_3[col_num][idx]
        counter = counter + 1

    for idx in range(num_4):
        for col_num in range(9):
            train_dataset[col_num][counter] = AF_4[col_num][idx]
        counter = counter + 1
    
    return train_dataset

def naive_bayes_classifier(TSTA, test_num, TRNA, train_num):
    ACC_Count = 0
    for i in tqdm(range(test_num)):
        
        # Naive Bayes Classifier
        count_c1 = 0
        count_c2 = 0
        for j in range(train_num):

            # (1) 計算 p(c)    
            if (TRNA[8][j] == 0): count_c1 = count_c1 + 1
            if (TRNA[8][j] == 1): count_c2 = count_c2 + 1
            
        prob_c1 = count_c1 / train_num
        prob_c2 = count_c2 / train_num

        #for j in range(train_num):
        
        # (2) 計算 p(x,c)
        cond_prob_c1=1
        cond_prob_c2=1

        for k in range(8):
            target_count_c1 = 0
            target_count_c2 = 0

            for ii in range(train_num):

                if   (TRNA[8][ii] == 0) and (TSTA[k][i] == TRNA[k][ii]): 
                    target_count_c1=target_count_c1+1

                elif (TRNA[8][ii] == 1) and (TSTA[k][i] == TRNA[k][ii]):
                    target_count_c2=target_count_c2+1

            cond_prob_c1=cond_prob_c1*target_count_c1/count_c1
            cond_prob_c2=cond_prob_c2*target_count_c2/count_c2

        # (3) 比較 2 個 score 看誰最高
        score_c1=prob_c1*cond_prob_c1
        score_c2=prob_c2*cond_prob_c2

        score_list = [score_c1, score_c2]

        # (4) 看看最高的那個類別有沒有和 y_test 一致
        max_score, max_position = find_max_pos(score_list, 2)

        if   (TSTA[8][i] == 0) and (max_position == 0):
            ACC_Count = ACC_Count + 1
        elif (TSTA[8][i] == 1) and (max_position == 1):
            ACC_Count = ACC_Count + 1
        else:
            ACC_Count = ACC_Count

    return ACC_Count

def KNN_classifier(TSTA, test_num, TRNA, train_num, mode):
    ACC_Count = 0
    
    for i in tqdm(range(test_num)):
        
        # K Nearest Neighbor
        test_instance = [TSTA[0][i], TSTA[1][i], TSTA[2][i], TSTA[3][i], TSTA[4][i], TSTA[5][i], TSTA[6][i], TSTA[7][i]]
        # Create Diff List
        diff_list = []
        for idx in range(train_num):

            diff_0 = (test_instance[0] - TRNA[0][idx])**2
            diff_1 = (test_instance[1] - TRNA[1][idx])**2
            diff_2 = (test_instance[2] - TRNA[2][idx])**2
            diff_3 = (test_instance[3] - TRNA[3][idx])**2
            diff_4 = (test_instance[4] - TRNA[4][idx])**2
            diff_5 = (test_instance[5] - TRNA[5][idx])**2
            diff_6 = (test_instance[6] - TRNA[6][idx])**2
            diff_7 = (test_instance[7] - TRNA[7][idx])**2

            total_diff = diff_0 + diff_1 + diff_2 + diff_3 + diff_4 + diff_5 + diff_6 + diff_7
            diff_list.append(total_diff)
        
        # Find 5 min diff idx
        skip_list=[-1, -1, -1, -1, -1]    
        min_1_idx, skip_list = find_min_pos(diff_list, train_num, skip_list, 5)
        min_2_idx, skip_list = find_min_pos(diff_list, train_num, skip_list, 5)
        min_3_idx, skip_list = find_min_pos(diff_list, train_num, skip_list, 5)
        min_4_idx, skip_list = find_min_pos(diff_list, train_num, skip_list, 5)
        min_5_idx, skip_list = find_min_pos(diff_list, train_num, skip_list, 5)
        
        if mode == "Majority_Voting":

            vote_1 = TRNA[8][min_1_idx]
            vote_2 = TRNA[8][min_2_idx]
            vote_3 = TRNA[8][min_3_idx]
            vote_4 = TRNA[8][min_4_idx]
            vote_5 = TRNA[8][min_5_idx]
            
            candidate_0 = 0
            candidate_1 = 0
            
            if vote_1 == 0:
                candidate_0 = candidate_0 + 1
            else:
                candidate_1 = candidate_1 + 1
            
            if vote_2 == 0:
                candidate_0 = candidate_0 + 1
            else:
                candidate_1 = candidate_1 + 1
                
            if vote_3 == 0:
                candidate_0 = candidate_0 + 1
            else:
                candidate_1 = candidate_1 + 1
            
            if vote_4 == 0:
                candidate_0 = candidate_0 + 1
            else:
                candidate_1 = candidate_1 + 1
            
            if vote_5 == 0:
                candidate_0 = candidate_0 + 1
            else:
                candidate_1 = candidate_1 + 1
            
            if candidate_0 > candidate_1:
                pred_ans = 0
            else:
                pred_ans = 1
            
            if pred_ans == TSTA[8][i]:
                ACC_Count = ACC_Count + 1
            else:
                ACC_Count = ACC_Count
                
        elif mode == "Weighted_Voting":
            
            vote_1 = TRNA[8][min_1_idx]
            vote_2 = TRNA[8][min_2_idx]
            vote_3 = TRNA[8][min_3_idx]
            vote_4 = TRNA[8][min_4_idx]
            vote_5 = TRNA[8][min_5_idx]
            diff_1 = diff_list[min_1_idx]
            diff_2 = diff_list[min_2_idx]
            diff_3 = diff_list[min_3_idx]
            diff_4 = diff_list[min_4_idx]
            diff_5 = diff_list[min_5_idx]
            candidate_0 = 0
            candidate_1 = 0
            
            if vote_1 == 0:
                candidate_0 = candidate_0 + 1/(diff_1**2)
            else:
                candidate_1 = candidate_1 + 1/(diff_1**2)
            
            if vote_2 == 0:
                candidate_0 = candidate_0 + 1/(diff_2**2)
            else:
                candidate_1 = candidate_1 + 1/(diff_2**2)
            
            if vote_3 == 0:
                candidate_0 = candidate_0 + 1/(diff_3**2)
            else:
                candidate_1 = candidate_1 + 1/(diff_3**2)
            
            if vote_4 == 0:
                candidate_0 = candidate_0 + 1/(diff_4**2)
            else:
                candidate_1 = candidate_1 + 1/(diff_4**2)
            
            if vote_5 == 0:
                candidate_0 = candidate_0 + 1/(diff_5**2)
            else:
                candidate_1 = candidate_1 + 1/(diff_5**2)
            
            if candidate_0 > candidate_1:
                pred_ans = 0
            else:
                pred_ans = 1
            
            if pred_ans == TSTA[8][i]:
                ACC_Count = ACC_Count + 1
            else:
                ACC_Count = ACC_Count
                
    return ACC_Count

# Model1: Equal-width + NBC
## Read Data

In [218]:
f = open("../pima.txt")
DATA = f.readlines()
DATA_COUNT = len(DATA)
f.close()

# 觀察到 DATA 目前為 string 形式，透過 split 拆解 string，並存入 DATA_NEW
# 另外，想要將 col 和 row 的位置對調

attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8 = [], [], [], [], [], [], [], [], []

for row in DATA:

    row_list = row.split(",")
    
    attr_0.append(float(row_list[0]))   
    attr_1.append(float(row_list[1]))  
    attr_2.append(float(row_list[2]))  
    attr_3.append(float(row_list[3]))
    attr_4.append(float(row_list[4]))
    attr_5.append(float(row_list[5]))  
    attr_6.append(float(row_list[6]))  
    attr_7.append(float(row_list[7]))
    attr_8.append(int(row_list[8]))

# 我只要透過 Attributes[0] 就能取得 DATA 第一個 Attribute 的所有值（預計有768個）
Attributes = [attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8]

In [242]:
# for i in range(768):
#     print(Attributes[0][i], Attributes[1][i], Attributes[2][i], Attributes[3][i], Attributes[4][i], Attributes[5][i], Attributes[6][i], Attributes[7][i], Attributes[8][i])

6.0 148.0 72.0 35.0 0.0 33.6 0.627 50.0 1
1.0 85.0 66.0 29.0 0.0 26.6 0.351 31.0 0
8.0 183.0 64.0 0.0 0.0 23.3 0.672 32.0 1
1.0 89.0 66.0 23.0 94.0 28.1 0.167 21.0 0
0.0 137.0 40.0 35.0 168.0 43.1 2.288 33.0 1
5.0 116.0 74.0 0.0 0.0 25.6 0.201 30.0 0
3.0 78.0 50.0 32.0 88.0 31.0 0.248 26.0 1
10.0 115.0 0.0 0.0 0.0 35.3 0.134 29.0 0
2.0 197.0 70.0 45.0 543.0 30.5 0.158 53.0 1
8.0 125.0 96.0 0.0 0.0 0.0 0.232 54.0 1
4.0 110.0 92.0 0.0 0.0 37.6 0.191 30.0 0
10.0 168.0 74.0 0.0 0.0 38.0 0.537 34.0 1
10.0 139.0 80.0 0.0 0.0 27.1 1.441 57.0 0
1.0 189.0 60.0 23.0 846.0 30.1 0.398 59.0 1
5.0 166.0 72.0 19.0 175.0 25.8 0.587 51.0 1
7.0 100.0 0.0 0.0 0.0 30.0 0.484 32.0 1
0.0 118.0 84.0 47.0 230.0 45.8 0.551 31.0 1
7.0 107.0 74.0 0.0 0.0 29.6 0.254 31.0 1
1.0 103.0 30.0 38.0 83.0 43.3 0.183 33.0 0
1.0 115.0 70.0 30.0 96.0 34.6 0.529 32.0 1
3.0 126.0 88.0 41.0 235.0 39.3 0.704 27.0 0
8.0 99.0 84.0 0.0 0.0 35.4 0.388 50.0 0
7.0 196.0 90.0 0.0 0.0 39.8 0.451 41.0 1
9.0 119.0 80.0 35.0 0.0 29.0 0.26

0.0 91.0 68.0 32.0 210.0 39.9 0.381 25.0 0
2.0 119.0 0.0 0.0 0.0 19.6 0.832 72.0 0
2.0 100.0 54.0 28.0 105.0 37.8 0.498 24.0 0
14.0 175.0 62.0 30.0 0.0 33.6 0.212 38.0 1
1.0 135.0 54.0 0.0 0.0 26.7 0.687 62.0 0
5.0 86.0 68.0 28.0 71.0 30.2 0.364 24.0 0
10.0 148.0 84.0 48.0 237.0 37.6 1.001 51.0 1
9.0 134.0 74.0 33.0 60.0 25.9 0.46 81.0 0
9.0 120.0 72.0 22.0 56.0 20.8 0.733 48.0 0
1.0 71.0 62.0 0.0 0.0 21.8 0.416 26.0 0
8.0 74.0 70.0 40.0 49.0 35.3 0.705 39.0 0
5.0 88.0 78.0 30.0 0.0 27.6 0.258 37.0 0
10.0 115.0 98.0 0.0 0.0 24.0 1.022 34.0 0
0.0 124.0 56.0 13.0 105.0 21.8 0.452 21.0 0
0.0 74.0 52.0 10.0 36.0 27.8 0.269 22.0 0
0.0 97.0 64.0 36.0 100.0 36.8 0.6 25.0 0
8.0 120.0 0.0 0.0 0.0 30.0 0.183 38.0 1
6.0 154.0 78.0 41.0 140.0 46.1 0.571 27.0 0
1.0 144.0 82.0 40.0 0.0 41.3 0.607 28.0 0
0.0 137.0 70.0 38.0 0.0 33.2 0.17 22.0 0
0.0 119.0 66.0 27.0 0.0 38.8 0.259 22.0 0
7.0 136.0 90.0 0.0 0.0 29.9 0.21 50.0 0
4.0 114.0 64.0 0.0 0.0 28.9 0.126 24.0 0
0.0 137.0 84.0 27.0 0.0 27.3 0.231 

# Model1: Equal-width + NBC

## Equal-Width with 10 bins

In [243]:
Attributes_EWD = []
for i in range(9):
    
    specific_col = Attributes[i]
    
    if i != 8:
        number_of_intervals = 10
        max_num_in_col = find_max(specific_col, DATA_COUNT)
        min_num_in_col = find_min(specific_col, DATA_COUNT)
        width = (max_num_in_col - min_num_in_col)/number_of_intervals

        # Decide 9 Split points, so that 10 intervals will form
        split_point_1 = max_num_in_col - 1 * width
        split_point_2 = max_num_in_col - 2 * width
        split_point_3 = max_num_in_col - 3 * width
        split_point_4 = max_num_in_col - 4 * width
        split_point_5 = max_num_in_col - 5 * width
        split_point_6 = max_num_in_col - 6 * width
        split_point_7 = max_num_in_col - 7 * width
        split_point_8 = max_num_in_col - 8 * width
        split_point_9 = max_num_in_col - 9 * width

        # transfer attributes into proper interval
        discrete_attr = []
        for j in range(DATA_COUNT):
            discrete_attr.append(-100)

        for j in range(DATA_COUNT):

            num = specific_col[j]

            if (num >= split_point_1): discrete_attr[j] = 0
            elif (num >= split_point_2) and (num < split_point_1): discrete_attr[j] = 1
            elif (num >= split_point_3) and (num < split_point_2): discrete_attr[j] = 2
            elif (num >= split_point_4) and (num < split_point_3): discrete_attr[j] = 3
            elif (num >= split_point_5) and (num < split_point_4): discrete_attr[j] = 4
            elif (num >= split_point_6) and (num < split_point_5): discrete_attr[j] = 5
            elif (num >= split_point_7) and (num < split_point_6): discrete_attr[j] = 6
            elif (num >= split_point_8) and (num < split_point_7): discrete_attr[j] = 7
            elif (num >= split_point_9) and (num < split_point_8): discrete_attr[j] = 8
            elif (num  < split_point_9): discrete_attr[j] = 9
                
        Attributes_EWD.append(discrete_attr)
        print(f"\n第 {str(i)} 個 Attribute")
        print(f"第一區間 [{str(round(split_point_1,3))} ~ max_num]")
        print(f"第二區間 [{str(round(split_point_2,3))} ~ {str(round(split_point_1,3))})")
        print(f"第三區間 [{str(round(split_point_3,3))} ~ {str(round(split_point_2,3))})")
        print(f"第四區間 [{str(round(split_point_4,3))} ~ {str(round(split_point_3,3))})")
        print(f"第五區間 [{str(round(split_point_5,3))} ~ {str(round(split_point_4,3))})")
        print(f"第六區間 [{str(round(split_point_6,3))} ~ {str(round(split_point_5,3))})")
        print(f"第七區間 [{str(round(split_point_7,3))} ~ {str(round(split_point_6,3))})")
        print(f"第八區間 [{str(round(split_point_8,3))} ~ {str(round(split_point_7,3))})")
        print(f"第九區間 [{str(round(split_point_9,3))} ~ {str(round(split_point_8,3))})")
        print(f"第十區間 [min_num ~ {str(round(split_point_9,3))})")
        
    else:
        Attributes_EWD.append(specific_col)


第 0 個 Attribute
第一區間 [15.3 ~ max_num]
第二區間 [13.6 ~ 15.3)
第三區間 [11.9 ~ 13.6)
第四區間 [10.2 ~ 11.9)
第五區間 [8.5 ~ 10.2)
第六區間 [6.8 ~ 8.5)
第七區間 [5.1 ~ 6.8)
第八區間 [3.4 ~ 5.1)
第九區間 [1.7 ~ 3.4)
第十區間 [min_num ~ 1.7)

第 1 個 Attribute
第一區間 [179.1 ~ max_num]
第二區間 [159.2 ~ 179.1)
第三區間 [139.3 ~ 159.2)
第四區間 [119.4 ~ 139.3)
第五區間 [99.5 ~ 119.4)
第六區間 [79.6 ~ 99.5)
第七區間 [59.7 ~ 79.6)
第八區間 [39.8 ~ 59.7)
第九區間 [19.9 ~ 39.8)
第十區間 [min_num ~ 19.9)

第 2 個 Attribute
第一區間 [109.8 ~ max_num]
第二區間 [97.6 ~ 109.8)
第三區間 [85.4 ~ 97.6)
第四區間 [73.2 ~ 85.4)
第五區間 [61.0 ~ 73.2)
第六區間 [48.8 ~ 61.0)
第七區間 [36.6 ~ 48.8)
第八區間 [24.4 ~ 36.6)
第九區間 [12.2 ~ 24.4)
第十區間 [min_num ~ 12.2)

第 3 個 Attribute
第一區間 [89.1 ~ max_num]
第二區間 [79.2 ~ 89.1)
第三區間 [69.3 ~ 79.2)
第四區間 [59.4 ~ 69.3)
第五區間 [49.5 ~ 59.4)
第六區間 [39.6 ~ 49.5)
第七區間 [29.7 ~ 39.6)
第八區間 [19.8 ~ 29.7)
第九區間 [9.9 ~ 19.8)
第十區間 [min_num ~ 9.9)

第 4 個 Attribute
第一區間 [761.4 ~ max_num]
第二區間 [676.8 ~ 761.4)
第三區間 [592.2 ~ 676.8)
第四區間 [507.6 ~ 592.2)
第五區間 [423.0 ~ 507.6)
第六區間 [338.4 ~ 423.0)
第七區間 

In [244]:
# for i in range(768):
#     print(Attributes_EWD[0][i], Attributes_EWD[1][i], Attributes_EWD[2][i], Attributes_EWD[3][i], Attributes_EWD[4][i], Attributes_EWD[5][i], Attributes_EWD[6][i], Attributes_EWD[7][i], Attributes_EWD[8][i])

6 2 4 6 9 4 7 5 1
9 5 4 7 9 6 8 8 0
5 0 4 9 9 6 7 8 1
9 5 4 7 8 5 9 9 0
9 3 6 6 8 3 0 7 1
7 4 3 9 9 6 9 8 0
8 6 5 6 8 5 9 9 1
4 4 9 9 9 4 9 8 0
8 0 4 5 3 5 9 4 1
5 3 2 9 9 9 9 4 1
7 4 2 9 9 4 9 8 0
4 1 3 9 9 4 8 7 1
4 3 3 9 9 5 4 3 0
9 0 5 7 0 5 8 3 1
7 1 4 8 7 6 7 4 1
5 4 9 9 9 5 8 8 1
9 4 3 5 7 3 7 8 1
5 4 3 9 9 5 9 8 1
9 4 7 6 9 3 9 7 0
9 4 4 6 8 4 8 8 1
8 3 2 5 7 4 7 8 0
5 5 3 9 9 4 8 5 0
5 0 2 9 9 4 8 6 1
4 4 3 6 9 5 9 8 1
3 2 2 6 8 4 9 4 1
4 3 4 7 8 5 9 6 1
5 2 3 9 9 4 9 6 1
9 5 4 8 8 6 8 9 0
2 2 3 8 8 6 9 3 0
7 4 2 9 9 4 8 7 0
7 4 3 7 9 4 8 3 0
8 2 3 6 7 5 6 8 1
8 5 5 8 9 6 9 9 0
6 5 2 9 9 7 9 8 0
4 3 3 6 9 5 8 5 0
7 4 5 6 7 6 6 7 0
3 3 3 9 9 5 8 7 0
4 4 3 6 9 5 7 5 1
8 5 4 5 9 4 8 8 1
7 4 4 5 7 4 4 4 1
8 0 4 7 9 4 9 9 0
5 3 3 9 9 4 7 7 0
5 4 2 8 9 6 9 5 0
4 1 0 7 7 3 7 4 1
5 2 4 9 9 5 9 6 0
9 0 4 6 9 3 2 9 1
9 2 5 9 9 5 7 8 0
8 6 4 7 9 5 7 9 0
5 4 4 6 9 4 8 8 1
5 4 9 9 9 9 9 9 0
9 4 3 8 9 7 8 9 0
9 4 5 8 9 6 8 9 0
7 5 4 7 9 6 8 8 0
5 1 2 6 6 4 8 3 1
5 2 4 5 5 4 7 6 0
9 6 5 8 9 

# Model1: Equal-width + NBC
## Random Shuffle

In [276]:
import random

# ================ #
# get random index #
# ================ #
random_list = [] 
for _ in range(DATA_COUNT):
    random_list.append(random.random())

arr_sort, arr_ori_idx = sort_col(random_list, len(random_list))

# ============================= #
# Initialize Shuffle_Attributes #
# ============================= #

Shuffle_Attributes = init_attributes(DATA_COUNT)

# ==================================== #
# Put Attributes in Shuffle Attributes #
# ==================================== #
counter = 0
for _ in range(DATA_COUNT):
    ori_index = arr_ori_idx[counter]
    for i in range(9):
        Shuffle_Attributes[i][counter] = Attributes_EWD[i][ori_index]
    counter = counter + 1

In [277]:
# for i in range(768):
#     print(Shuffle_Attributes[0][i], Shuffle_Attributes[1][i], Shuffle_Attributes[2][i], Shuffle_Attributes[3][i], Shuffle_Attributes[4][i], Shuffle_Attributes[5][i], Shuffle_Attributes[6][i], Shuffle_Attributes[7][i], Shuffle_Attributes[8][i])

8 5 3 8 9 6 9 9 0
9 5 4 7 8 3 8 9 0
2 4 4 9 9 5 8 7 1
2 3 2 9 9 3 7 6 1
9 1 4 5 3 3 7 8 1
8 3 4 7 7 5 7 8 1
5 4 3 6 9 4 5 6 1
7 2 3 7 6 4 8 8 0
7 2 2 9 9 5 8 3 1
9 2 9 9 9 3 9 8 1
4 2 3 6 7 4 6 7 1
8 4 6 7 8 5 8 9 0
7 5 2 5 9 4 8 8 0
9 4 3 8 9 5 6 9 0
8 4 4 9 9 6 9 8 0
9 5 3 6 9 5 9 9 0
5 4 1 6 9 3 9 5 1
6 4 5 7 7 5 4 7 1
8 3 4 6 8 3 8 8 0
6 0 2 9 9 3 4 5 0
9 3 2 7 8 5 8 9 0
7 2 3 9 9 4 8 2 1
9 0 9 9 9 3 9 6 1
7 4 4 5 9 4 9 7 0
6 4 9 9 9 9 9 9 0
4 3 5 9 9 5 5 7 1
4 1 0 7 7 3 7 4 1
2 3 9 6 9 4 7 6 1
8 4 5 6 8 6 9 9 0
6 4 4 6 9 4 9 9 0
9 0 2 5 3 3 9 9 1
6 5 3 9 9 6 9 8 0
7 4 4 8 9 5 8 8 1
7 4 2 9 9 4 8 7 0
5 6 4 5 9 4 7 6 0
9 4 4 8 9 6 9 9 0
7 2 3 5 3 4 7 7 0
9 5 3 9 9 5 7 8 0
9 9 4 6 9 5 8 9 0
8 4 5 6 9 5 7 8 0
8 4 2 8 9 5 8 8 0
7 1 4 9 9 5 9 6 1
7 4 3 9 9 5 9 5 0
8 5 4 8 9 5 7 9 0
9 4 4 8 9 6 7 8 0
8 4 2 5 8 4 9 8 0
4 4 3 6 7 4 9 7 1
8 4 3 4 8 4 9 9 0
5 2 4 9 9 5 9 6 0
5 3 3 9 9 6 8 2 0
8 2 3 8 9 5 9 2 0
9 3 4 8 7 6 9 9 0
4 1 4 9 9 4 9 7 0
6 3 4 9 9 5 8 8 1
7 1 4 8 8 5 8 7 1
8 5 3 9 9 

# Model1: Equal-width + NBC
## Split into 5 folds

In [255]:
# ================== #
# Initialize 5 folds #
# ================== #

AF_1 = init_attributes(153)
AF_2 = init_attributes(153)
AF_3 = init_attributes(153)
AF_4 = init_attributes(153)
AF_5 = init_attributes(156)

# ================== #
# Split into 5 folds #
# ================== #

counter_k1 = 0
counter_k2 = 0
counter_k3 = 0
counter_k4 = 0
counter_k5 = 0

for i in range(DATA_COUNT):
    if i < 153*1:
        for j in range(9):
            AF_1[j][counter_k1] = Shuffle_Attributes[j][i]
        counter_k1 = counter_k1 + 1
        
    elif i < 153*2 and i >= 153*1:
        for j in range(9):
            AF_2[j][counter_k2] = Shuffle_Attributes[j][i]
        counter_k2 = counter_k2 + 1
        
    elif i < 153*3 and i >= 153*2:
        for j in range(9):
            AF_3[j][counter_k3] = Shuffle_Attributes[j][i]
        counter_k3 = counter_k3 + 1

    elif i < 153*4 and i >= 153*3:
        for j in range(9):
            AF_4[j][counter_k4] = Shuffle_Attributes[j][i]
        counter_k4 = counter_k4 + 1
        
    elif i >= 153*4:
        for j in range(9):
            AF_5[j][counter_k5] = Shuffle_Attributes[j][i]
        counter_k5 = counter_k5 + 1

# Model1: Equal-width + NBC
## Naive Baysian Classifier

In [256]:
# ================================= #
# Attributes_fold_1 當 test dataset #
# ================================= #
test_dataset = AF_1
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_2, AF_3, AF_4, AF_5)
fold_1_correct = naive_bayes_classifier(test_dataset, 153, train_dataset, 615)

# ================================= #
# Attributes_fold_2 當 test dataset #
# ================================= #
test_dataset = AF_2
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_1, AF_3, AF_4, AF_5)
fold_2_correct = naive_bayes_classifier(test_dataset, 153, train_dataset, 615)

# ================================= #
# Attributes_fold_3 當 test dataset #
# ================================= #
test_dataset = AF_3
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_1, AF_2, AF_4, AF_5)
fold_3_correct = naive_bayes_classifier(test_dataset, 153, train_dataset, 615)

# ================================= #
# Attributes_fold_4 當 test dataset #
# ================================= #
test_dataset = AF_4
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_1, AF_2, AF_3, AF_5)
fold_4_correct = naive_bayes_classifier(test_dataset, 153, train_dataset, 615)

# ================================= #
# Attributes_fold_5 當 test dataset #
# ================================= #
test_dataset = AF_5
train_dataset= init_attributes(153*4)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 153, AF_1, AF_2, AF_3, AF_4)
fold_5_correct = naive_bayes_classifier(test_dataset, 156, train_dataset, 153*4)

100%|██████████████████████████████████████████████████████████████████████████████| 156/156 [00:00<00:00, 1147.13it/s]


# Model1: Equal-width + NBC
## Calculate Average Accuracy

In [257]:
print(fold_1_correct, fold_2_correct, fold_3_correct, fold_4_correct, fold_5_correct)
P1_BAR = (fold_1_correct + fold_2_correct + fold_3_correct + fold_4_correct + fold_5_correct) / DATA_COUNT
print(f"[Model 1: Equal-width Discretization + Naive Bayes Classifier] Average is {P1_BAR}")

123 112 114 118 116
[Model 1: Equal-width Discretization + Naive Bayes Classifier] Average is 0.7591145833333334


# Model2: Equal-frequency + NBC
## Read Data

In [258]:
f = open("../pima.txt")
DATA = f.readlines()
DATA_COUNT = len(DATA)
f.close()

# 觀察到 DATA 目前為 string 形式，透過 split 拆解 string，並存入 DATA_NEW
# 另外，想要將 col 和 row 的位置對調

attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8 = [], [], [], [], [], [], [], [], []

for row in DATA:

    row_list = row.split(",")
    
    attr_0.append(float(row_list[0]))   
    attr_1.append(float(row_list[1]))  
    attr_2.append(float(row_list[2]))  
    attr_3.append(float(row_list[3]))
    attr_4.append(float(row_list[4]))
    attr_5.append(float(row_list[5]))  
    attr_6.append(float(row_list[6]))  
    attr_7.append(float(row_list[7]))
    attr_8.append(int(row_list[8]))

# 我只要透過 Attributes[0] 就能取得 DATA 第一個 Attribute 的所有值（預計有768個）
Attributes = [attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8]

# Model2: Equal-frequency + NBC
## Equal-Frequency with 10 Bins

In [278]:
int(number_of_attr_in_interval)

76

In [259]:
Attributes_Sort = []

for i in range(9):
    if i < 8:
        col, _ = sort_col(Attributes[i], DATA_COUNT)
        Attributes_Sort.append(col)
    else:
        Attributes_Sort.append(Attributes[i])

##############
# Read Again #
##############
f = open("../pima.txt")
DATA = f.readlines()
DATA_COUNT = len(DATA)
f.close()

number_of_intervals = 10
number_of_attr_in_interval = DATA_COUNT / number_of_intervals

attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8 = [], [], [], [], [], [], [], [], []

for row in DATA:

    row_list = row.split(",")
    
    attr_0.append(float(row_list[0]))   
    attr_1.append(float(row_list[1]))  
    attr_2.append(float(row_list[2]))  
    attr_3.append(float(row_list[3]))
    attr_4.append(float(row_list[4]))
    attr_5.append(float(row_list[5]))  
    attr_6.append(float(row_list[6]))  
    attr_7.append(float(row_list[7]))
    attr_8.append(int(row_list[8]))
    

# 我只要透過 Attributes[0] 就能取得 DATA 第一個 Attribute 的所有值（預計有768個）
Attributes = [attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8]


Attributes_EFD = []

for j in range(9):
    specific_col_sort = Attributes_Sort[j]
    specific_col = Attributes[j]
        
    if j != 8:

        split_point_1 = specific_col_sort[1 * int(number_of_attr_in_interval)]
        split_point_2 = specific_col_sort[2 * int(number_of_attr_in_interval)]
        split_point_3 = specific_col_sort[3 * int(number_of_attr_in_interval)]
        split_point_4 = specific_col_sort[4 * int(number_of_attr_in_interval)]
        split_point_5 = specific_col_sort[5 * int(number_of_attr_in_interval)]
        split_point_6 = specific_col_sort[6 * int(number_of_attr_in_interval)]
        split_point_7 = specific_col_sort[7 * int(number_of_attr_in_interval)]
        split_point_8 = specific_col_sort[8 * int(number_of_attr_in_interval)]
        split_point_9 = specific_col_sort[9 * int(number_of_attr_in_interval)]

        # transfer attributes into proper interval
        discrete_attr = []
        for i in range(DATA_COUNT):
            discrete_attr.append(-100)

        for i in range(DATA_COUNT):

            num = specific_col[i]

            if num >= split_point_1: discrete_attr[i] = 0
            elif num >= split_point_2 and num < split_point_1: discrete_attr[i] = 1
            elif num >= split_point_3 and num < split_point_2: discrete_attr[i] = 2
            elif num >= split_point_4 and num < split_point_3: discrete_attr[i] = 3
            elif num >= split_point_5 and num < split_point_4: discrete_attr[i] = 4
            elif num >= split_point_6 and num < split_point_5: discrete_attr[i] = 5
            elif num >= split_point_7 and num < split_point_6: discrete_attr[i] = 6
            elif num >= split_point_8 and num < split_point_7: discrete_attr[i] = 7
            elif num >= split_point_9 and num < split_point_8: discrete_attr[i] = 8
            elif num < split_point_9: discrete_attr[i] = 9
                
        Attributes_EFD.append(discrete_attr)
        print(f"\n第 {str(j)} 個 Attribute")
        print(f"第一區間 [{str(round(split_point_1,3))} ~ max_num]")
        print(f"第二區間 [{str(round(split_point_2,3))} ~ {str(round(split_point_1,3))})")
        print(f"第三區間 [{str(round(split_point_3,3))} ~ {str(round(split_point_2,3))})")
        print(f"第四區間 [{str(round(split_point_4,3))} ~ {str(round(split_point_3,3))})")
        print(f"第五區間 [{str(round(split_point_5,3))} ~ {str(round(split_point_4,3))})")
        print(f"第六區間 [{str(round(split_point_6,3))} ~ {str(round(split_point_5,3))})")
        print(f"第七區間 [{str(round(split_point_7,3))} ~ {str(round(split_point_6,3))})")
        print(f"第八區間 [{str(round(split_point_8,3))} ~ {str(round(split_point_7,3))})")
        print(f"第九區間 [{str(round(split_point_9,3))} ~ {str(round(split_point_8,3))})")
        print(f"第十區間 [min_num ~ {str(round(split_point_9,3))})")
    else:
        Attributes_EFD.append(specific_col)


第 0 個 Attribute
第一區間 [9.0 ~ max_num]
第二區間 [7.0 ~ 9.0)
第三區間 [5.0 ~ 7.0)
第四區間 [4.0 ~ 5.0)
第五區間 [3.0 ~ 4.0)
第六區間 [2.0 ~ 3.0)
第七區間 [1.0 ~ 2.0)
第八區間 [1.0 ~ 1.0)
第九區間 [0.0 ~ 1.0)
第十區間 [min_num ~ 0.0)

第 1 個 Attribute
第一區間 [167.0 ~ max_num]
第二區間 [147.0 ~ 167.0)
第三區間 [135.0 ~ 147.0)
第四區間 [125.0 ~ 135.0)
第五區間 [117.0 ~ 125.0)
第六區間 [109.0 ~ 117.0)
第七區間 [102.0 ~ 109.0)
第八區間 [95.0 ~ 102.0)
第九區間 [87.0 ~ 95.0)
第十區間 [min_num ~ 87.0)

第 2 個 Attribute
第一區間 [88.0 ~ max_num]
第二區間 [82.0 ~ 88.0)
第三區間 [78.0 ~ 82.0)
第四區間 [74.0 ~ 78.0)
第五區間 [72.0 ~ 74.0)
第六區間 [68.0 ~ 72.0)
第七區間 [64.0 ~ 68.0)
第八區間 [62.0 ~ 64.0)
第九區間 [54.0 ~ 62.0)
第十區間 [min_num ~ 54.0)

第 3 個 Attribute
第一區間 [40.0 ~ max_num]
第二區間 [35.0 ~ 40.0)
第三區間 [31.0 ~ 35.0)
第四區間 [27.0 ~ 31.0)
第五區間 [23.0 ~ 27.0)
第六區間 [18.0 ~ 23.0)
第七區間 [10.0 ~ 18.0)
第八區間 [0.0 ~ 10.0)
第九區間 [0.0 ~ 0.0)
第十區間 [min_num ~ 0.0)

第 4 個 Attribute
第一區間 [210.0 ~ max_num]
第二區間 [152.0 ~ 210.0)
第三區間 [108.0 ~ 152.0)
第四區間 [74.0 ~ 108.0)
第五區間 [36.0 ~ 74.0)
第六區間 [0.0 ~ 36.0)
第七區間 [0.0 ~ 0.0)


# Model2: Equal-frequency + NBC
## Random Shuffle

In [260]:
import random

# ================ #
# get random index #
# ================ #
# random_list = [] 
# for _ in range(DATA_COUNT):
#     random_list.append(random.random())

# arr_sort, arr_ori_idx = sort_col(random_list, len(random_list))

# ============================= #
# Initialize Shuffle_Attributes #
# ============================= #

Shuffle_Attributes = init_attributes(DATA_COUNT)

# ==================================== #
# Put Attributes in Shuffle Attributes #
# ==================================== #
counter = 0
for _ in range(DATA_COUNT):
    ori_index = arr_ori_idx[counter]
    for i in range(9):
        Shuffle_Attributes[i][counter] = Attributes_EFD[i][ori_index]
    counter = counter + 1

# Model2: Equal-Frequency + NBC
## Split into 5 folds

In [261]:
# ================== #
# Initialize 5 folds #
# ================== #

AF_1 = init_attributes(153)
AF_2 = init_attributes(153)
AF_3 = init_attributes(153)
AF_4 = init_attributes(153)
AF_5 = init_attributes(156)

# ================== #
# Split into 5 folds #
# ================== #

counter_k1 = 0
counter_k2 = 0
counter_k3 = 0
counter_k4 = 0
counter_k5 = 0

for i in range(DATA_COUNT):
    if i < 153*1:
        for j in range(9):
            AF_1[j][counter_k1] = Shuffle_Attributes[j][i]
        counter_k1 = counter_k1 + 1
        
    elif i < 153*2 and i >= 153*1:
        for j in range(9):
            AF_2[j][counter_k2] = Shuffle_Attributes[j][i]
        counter_k2 = counter_k2 + 1
        
    elif i < 153*3 and i >= 153*2:
        for j in range(9):
            AF_3[j][counter_k3] = Shuffle_Attributes[j][i]
        counter_k3 = counter_k3 + 1

    elif i < 153*4 and i >= 153*3:
        for j in range(9):
            AF_4[j][counter_k4] = Shuffle_Attributes[j][i]
        counter_k4 = counter_k4 + 1
        
    elif i >= 153*4:
        for j in range(9):
            AF_5[j][counter_k5] = Shuffle_Attributes[j][i]
        counter_k5 = counter_k5 + 1

# Model2: Equal-Frequency + NBC
## Naive Baysian Classifier

In [262]:
# ================================= #
# Attributes_fold_1 當 test dataset #
# ================================= #
test_dataset = AF_1
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_2, AF_3, AF_4, AF_5)
fold_1_correct = naive_bayes_classifier(test_dataset, 153, train_dataset, 615)

# ================================= #
# Attributes_fold_2 當 test dataset #
# ================================= #
test_dataset = AF_2
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_1, AF_3, AF_4, AF_5)
fold_2_correct = naive_bayes_classifier(test_dataset, 153, train_dataset, 615)

# ================================= #
# Attributes_fold_3 當 test dataset #
# ================================= #
test_dataset = AF_3
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_1, AF_2, AF_4, AF_5)
fold_3_correct = naive_bayes_classifier(test_dataset, 153, train_dataset, 615)

# ================================= #
# Attributes_fold_4 當 test dataset #
# ================================= #
test_dataset = AF_4
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_1, AF_2, AF_3, AF_5)
fold_4_correct = naive_bayes_classifier(test_dataset, 153, train_dataset, 615)

# ================================= #
# Attributes_fold_5 當 test dataset #
# ================================= #
test_dataset = AF_5
train_dataset= init_attributes(153*4)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 153, AF_1, AF_2, AF_3, AF_4)
fold_5_correct = naive_bayes_classifier(test_dataset, 156, train_dataset, 153*4)

100%|██████████████████████████████████████████████████████████████████████████████| 156/156 [00:00<00:00, 1155.42it/s]


In [263]:
print(fold_1_correct, fold_2_correct, fold_3_correct, fold_4_correct, fold_5_correct)

P2_BAR = (fold_1_correct + fold_2_correct + fold_3_correct + fold_4_correct + fold_5_correct) / DATA_COUNT
print(f"[Model 2: Equal-frequency Discretization + Naive Bayes Classifier] Average is {P2_BAR}")

118 112 111 119 116
[Model 2: Equal-frequency Discretization + Naive Bayes Classifier] Average is 0.75


In [264]:
import math

print("Compare Model 1 and Model 2.")
print("H0: P1_BAR = P2_BAR")

P_AVG = (P1_BAR + P2_BAR) / 2

Z = (P1_BAR-P2_BAR)/math.sqrt(2/DATA_COUNT*P_AVG*(1-P_AVG))

print(f"(Two-tailed test with alpha = 5%) Z = {Z}")
if Z < -1.96 or Z > 1.96:
    print("H0 is rejected.")
else:
    print("H0 is NOT rejected.")

Compare Model 1 and Model 2.
H0: P1_BAR = P2_BAR
(Two-tailed test with alpha = 5%) Z = 0.4150316861552657
H0 is NOT rejected.


# Model 3: KNN + Majority Voting
## Read Data

In [265]:
f = open("../pima.txt")
DATA = f.readlines()
DATA_COUNT = len(DATA)
f.close()

# 觀察到 DATA 目前為 string 形式，透過 split 拆解 string，並存入 DATA_NEW
# 另外，想要將 col 和 row 的位置對調

attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8 = [], [], [], [], [], [], [], [], []

for row in DATA:

    row_list = row.split(",")
    
    attr_0.append(float(row_list[0]))
    attr_1.append(float(row_list[1]))  
    attr_2.append(float(row_list[2]))  
    attr_3.append(float(row_list[3]))
    attr_4.append(float(row_list[4]))
    attr_5.append(float(row_list[5]))  
    attr_6.append(float(row_list[6]))  
    attr_7.append(float(row_list[7]))
    attr_8.append(int(row_list[8]))

# 我只要透過 Attributes[0] 就能取得 DATA 第一個 Attribute 的所有值（預計有768個）
Attributes = [attr_0, attr_1, attr_2, attr_3, attr_4, attr_5, attr_6, attr_7, attr_8]

# Model 3: KNN + Majority Voting
## Normalize Data

In [266]:
for num in range(DATA_COUNT):
    Attributes_Norm = []
    for col_num in range(8):
        max_attr = find_max(Attributes[col_num], DATA_COUNT)
        min_attr = find_min(Attributes[col_num], DATA_COUNT)
        attr_norm = []
        for num in range(DATA_COUNT):
            attr_norm.append((Attributes[col_num][num]-min_attr)/(max_attr-min_attr))
        Attributes_Norm.append(attr_norm)

# Model 3: KNN + Majority Voting
## Random Shuffle

In [267]:
import random

# ================ #
# get random index #
# ================ #
# random_list = [] 
# for _ in range(DATA_COUNT):
#     random_list.append(random.random())

# arr_sort, arr_ori_idx = sort_col(random_list, len(random_list))

# ============================= #
# Initialize Shuffle_Attributes #
# ============================= #

Shuffle_Attributes = init_attributes(DATA_COUNT)

# ==================================== #
# Put Attributes in Shuffle Attributes #
# ==================================== #
counter = 0
for _ in range(DATA_COUNT):
    ori_index = arr_ori_idx[counter]
    for i in range(8):
        Shuffle_Attributes[i][counter] = Attributes_Norm[i][ori_index]
    Shuffle_Attributes[8][counter] = Attributes[8][ori_index]
    counter = counter + 1

# Model 3: KNN + Majority Voting
## Split into 5 folds

In [268]:
# ================== #
# Initialize 5 folds #
# ================== #

AF_1 = init_attributes(153)
AF_2 = init_attributes(153)
AF_3 = init_attributes(153)
AF_4 = init_attributes(153)
AF_5 = init_attributes(156)

# ================== #
# Split into 5 folds #
# ================== #

counter_k1 = 0
counter_k2 = 0
counter_k3 = 0
counter_k4 = 0
counter_k5 = 0

for i in range(DATA_COUNT):
    if i < 153*1:
        for j in range(9):
            AF_1[j][counter_k1] = Shuffle_Attributes[j][i]
        counter_k1 = counter_k1 + 1
        
    elif i < 153*2 and i >= 153*1:
        for j in range(9):
            AF_2[j][counter_k2] = Shuffle_Attributes[j][i]
        counter_k2 = counter_k2 + 1
        
    elif i < 153*3 and i >= 153*2:
        for j in range(9):
            AF_3[j][counter_k3] = Shuffle_Attributes[j][i]
        counter_k3 = counter_k3 + 1

    elif i < 153*4 and i >= 153*3:
        for j in range(9):
            AF_4[j][counter_k4] = Shuffle_Attributes[j][i]
        counter_k4 = counter_k4 + 1
        
    elif i >= 153*4:
        for j in range(9):
            AF_5[j][counter_k5] = Shuffle_Attributes[j][i]
        counter_k5 = counter_k5 + 1

# Model 3: KNN + Majority Voting
## KNN

In [269]:
# ================================= #
# Attributes_fold_1 當 test dataset #
# ================================= #
test_dataset = AF_1
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_2, AF_3, AF_4, AF_5)
fold_1_correct = KNN_classifier(test_dataset, 153, train_dataset, 615, mode="Majority_Voting")

# ================================= #
# Attributes_fold_2 當 test dataset #
# ================================= #
test_dataset = AF_2
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_1, AF_3, AF_4, AF_5)
fold_2_correct = KNN_classifier(test_dataset, 153, train_dataset, 615, mode="Majority_Voting")

# ================================= #
# Attributes_fold_3 當 test dataset #
# ================================= #
test_dataset = AF_3
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_1, AF_2, AF_4, AF_5)
fold_3_correct = KNN_classifier(test_dataset, 153, train_dataset, 615, mode="Majority_Voting")

# ================================= #
# Attributes_fold_4 當 test dataset #
# ================================= #
test_dataset = AF_4
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_1, AF_2, AF_3, AF_5)
fold_4_correct = KNN_classifier(test_dataset, 153, train_dataset, 615, mode="Majority_Voting")

# ================================= #
# Attributes_fold_5 當 test dataset #
# ================================= #
test_dataset = AF_5
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 153, AF_1, AF_2, AF_3, AF_4)
fold_5_correct = KNN_classifier(test_dataset, 156, train_dataset, 153*4, mode="Majority_Voting")

100%|███████████████████████████████████████████████████████████████████████████████| 156/156 [00:00<00:00, 478.11it/s]


In [270]:
print(fold_1_correct, fold_2_correct, fold_3_correct, fold_4_correct, fold_5_correct)

P3_BAR = (fold_1_correct + fold_2_correct + fold_3_correct + fold_4_correct + fold_5_correct) / DATA_COUNT
print(f"[Model 3: K-Nearest Neighbor + Majority Voting] Average is {P3_BAR}")

107 112 117 109 112
[Model 3: K-Nearest Neighbor + Majority Voting] Average is 0.7252604166666666


# Model 4: KNN + Distance Weighted Voting

In [271]:
# ================================= #
# Attributes_fold_1 當 test dataset #
# ================================= #
test_dataset = AF_1
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_2, AF_3, AF_4, AF_5)
fold_1_correct = KNN_classifier(test_dataset, 153, train_dataset, 615, mode="Weighted_Voting")

# ================================= #
# Attributes_fold_2 當 test dataset #
# ================================= #
test_dataset = AF_2
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_1, AF_3, AF_4, AF_5)
fold_2_correct = KNN_classifier(test_dataset, 153, train_dataset, 615, mode="Weighted_Voting")

# ================================= #
# Attributes_fold_3 當 test dataset #
# ================================= #
test_dataset = AF_3
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_1, AF_2, AF_4, AF_5)
fold_3_correct = KNN_classifier(test_dataset, 153, train_dataset, 615, mode="Weighted_Voting")

# ================================= #
# Attributes_fold_4 當 test dataset #
# ================================= #
test_dataset = AF_4
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 156, AF_1, AF_2, AF_3, AF_5)
fold_4_correct = KNN_classifier(test_dataset, 153, train_dataset, 615, mode="Weighted_Voting")

# ================================= #
# Attributes_fold_5 當 test dataset #
# ================================= #
test_dataset = AF_5
train_dataset= init_attributes(153*3+156)
train_dataset= map_into_train_dataset(train_dataset, 153, 153, 153, 153, AF_1, AF_2, AF_3, AF_4)
fold_5_correct = KNN_classifier(test_dataset, 156, train_dataset, 153*4, mode="Weighted_Voting")

100%|███████████████████████████████████████████████████████████████████████████████| 156/156 [00:00<00:00, 491.20it/s]


In [272]:
print(fold_1_correct, fold_2_correct, fold_3_correct, fold_4_correct, fold_5_correct)

P4_BAR = (fold_1_correct + fold_2_correct + fold_3_correct + fold_4_correct + fold_5_correct) / DATA_COUNT
print(f"[Model 4: K-Nearest Neighbor + Distance-Weighted Voting] Average is {P4_BAR}")

110 112 118 111 109
[Model 4: K-Nearest Neighbor + Distance-Weighted Voting] Average is 0.7291666666666666


In [273]:
import math

print("Compare Model 3 and Model 4.")
print("H0: P3_BAR = P4_BAR")

P_AVG = (P3_BAR + P4_BAR) / 2

Z = (P3_BAR-P4_BAR)/math.sqrt(2/DATA_COUNT*P_AVG*(1-P_AVG))

print(f"(Two-tailed test with alpha = 5%) Z = {Z}")
if Z < -1.96 or Z > 1.96:
    print("H0 is rejected.")
else:
    print("H0 is NOT rejected.")

Compare Model 3 and Model 4.
H0: P3_BAR = P4_BAR
(Two-tailed test with alpha = 5%) Z = -0.17186334668355702
H0 is NOT rejected.


In [274]:
import math

print("Compare Model 2 and Model 3.")
print("H0: P2_BAR = P3_BAR")

P_AVG = (P2_BAR + P3_BAR) / 2

Z = (P2_BAR-P3_BAR)/math.sqrt(2/DATA_COUNT*P_AVG*(1-P_AVG))

print(f"(Two-tailed test with alpha = 5%) Z = {Z}")
if Z < -1.96 or Z > 1.96:
    print("H0 is rejected.")
else:
    print("H0 is NOT rejected.")

Compare Model 2 and Model 3.
H0: P2_BAR = P3_BAR
(Two-tailed test with alpha = 5%) Z = 1.1020003458100602
H0 is NOT rejected.
